
# Spaces and forms on subdomains (Unit 1.5)

In NGSolve, finite element spaces can be defined on subdomains. This is useful for multiphysics problems like fluid-structure interaction.

In addition, bilinear or linear forms can be defined as integrals over regions. Regions are parts of the domain. They may be subdomains, or parts of the domain boundary, or parts of subdomain interfaces.


In [1]:
import netgen.gui
%gui tk
import netgen.meshing as msh
from ngsolve import *
import ngsolve



### Naming subdomains and boundaries

We define a geometry with multiple regions and assign names to these regions below:


In [2]:
a=0
b=1
nel=10 
m = msh.Mesh()
m.dim = 1
pnums = []

for i in range(nel+1):
    pnums.append(m.Add (msh.MeshPoint(msh.Pnt(a+(b-a)*i/nel, 0, 0))))

for i in range(3):
    m.Add (msh.Element1D([pnums[i], pnums[i+1]],index=1))
for i in range(3,7):
    m.Add (msh.Element1D([pnums[i], pnums[i+1]],index=2))
for i in range(7,nel):
    m.Add (msh.Element1D([pnums[i], pnums[i+1]],index=1))

m.SetMaterial(1, 'outer')
m.SetMaterial(2,'inner')

# add points
m.Add (msh.Element0D (pnums[0], index=1))
m.Add (msh.Element0D (pnums[3], index=2))
m.Add (msh.Element0D (pnums[7], index=3))
m.Add (msh.Element0D (pnums[nel], index=4))

# set boundary condition names
m.SetBCName(0,'lout')
m.SetBCName(1,'lin')
m.SetBCName(2,'rin')
m.SetBCName(3,'rout')
m.Save('test.vol')

type(m)
mesh = ngsolve.Mesh(m)



- These statements define two rectangular subdomain regions named "inner", "outer".

- The left and right parts of the outer segment's boundaries define boundary regions, respectively labeled "lout", "rout".

- Similarly, the left and right parts of the inner rectangle's boundaries are regions named "lin", "rin".

- You can see the subdomain outlines in the Netgen window when you select the Geometry tab. When you select the Mesh tab and double click on a point, the two subdomains are rendered in different colors.

### A finite element space on a subdomain


In [3]:
fes1 = H1(mesh, definedon="inner")

u1 = GridFunction(fes1, "u1")
u1.Set((x-0.5)**2)
Draw(u1)


Note how $u_1$ is displayed only in the inner region.
### Integrating on regions

You have already seen how boundary regions are used in setting Dirichlet boundary conditions.


In [4]:
fes = H1(mesh, order=3, dirichlet="lout|lin|rout")

u = fes.TrialFunction()
v = fes.TestFunction()
gfu = GridFunction(fes)

Boundary regions or subdomains can also serve as domains of integration.

In [5]:
f = LinearForm(fes)
f += SymbolicLFI(u1*v, definedon=mesh.Materials("inner"))
f += SymbolicLFI(0.1*v, definedon=mesh.Boundaries("rin"))
f.Assemble()
# mesh.Materials("inner") and mesh.Boundaries("t") are "region" objects

Here the functional $f$ is defined as a sum of two integrals, one over the inner subdomain, and another over the top boundary:
$$f(v)\int_{\Omega_{inner}} u_1 v dx + \int_{\Gamma_{top}} \frac{v}{10} ds$$

Note: SymbolicLFI has no information about the mesh. Hence its not enough to give it the just the name of the region ("inner", or "t"). We must give it the region objects (named by the "inner" and "t" labels) using the definedon flag.

In [6]:
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()

# Solve the problem:
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu)


###  More about region objects¶

In [7]:
mesh.GetMaterials()    # list all subdomains

('outer', 'inner')

In [8]:
mesh.GetBoundaries()   # list boundary/interface regions

('lout', 'lin', 'rin', 'rout')

In [9]:
mesh.Materials("inner") # look at object's type 

In [11]:
mesh.Boundaries("rout")    # same type!  


### Operations with regions

Print region information:


In [12]:
print(mesh.Materials("inner").Mask())
print(mesh.Materials("[a-z]*").Mask())  # can use regexp
print(mesh.Boundaries('lin|rout').Mask())



0: 01
0: 11
0: 0101


Add regions:

In [13]:
io = mesh.Materials("inner") + mesh.Materials("outer")
print(io.Mask()) # add regions

0: 11


Take complement of a region:

In [14]:
c = ~mesh.Materials("inner") # complement of a region
print(c.Mask())

0: 10


Subtract regions:

In [15]:
diff = mesh.Materials("inner|outer") - mesh.Materials("outer")
print(diff.Mask()) # difference of two regions


0: 01


Set a piecewise constant CoefficientiFunction using the subdomains:

In [18]:
domain_values = {'inner': -1,  'outer': 1}
values_list = [domain_values[mat]
               for mat in mesh.GetMaterials()]
cf = CoefficientFunction(values_list)
Draw(cf, mesh, 'piecewise')
# so if you initialize a coefficient function with a list, it assumes that each element is for a domain

In [16]:
values_list

[1, 3.7]

Coefficients on boundary regions are given similarly. Let's make a linear function that equals 2 at the bottom right vertex of current domain (0,2) x (0,2) and equals zero at the remaining vertices.

In [20]:
bdry_values = {'lin': x, 'rin': 2-x}
values_list = [bdry_values[bc]
               if bc in bdry_values.keys() else 0
               for bc in mesh.GetBoundaries()]
cf = CoefficientFunction(values_list)
Draw(cf, mesh, 'piecewise')


Pitfall! Look at the GUI output. Is this what we expected?

What happened here? The object cf has no information on boundary regions, so it cannot associate the list of values to boundary regions. By default, the list of values are assumed to be subdomain values.

To associate these values to boundary regions, we use a GridFunction and Set, which also lets us view an extension of these boundary values into the domain.


In [21]:
g = GridFunction(H1(mesh), name='bdry')
g.Set(cf, definedon=~mesh.Boundaries('')) # complement of empty set gives all boundaries
Draw(g, max=2, min=0)




If you think that specifying the whole boundary using

~mesh.Boundaries('')

is convoluted, then you may use

g.Set(cf, definedon=mesh.Boundaries('b|r')).



In [22]:
g.Set(cf, definedon=mesh.Boundaries('lin|rin'))
Redraw()


What happens if you Set using b and then on r?

In [23]:
g.Set(cf, definedon=mesh.Boundaries('lin'))
g.Set(cf, definedon=mesh.Boundaries('rin'))
Redraw()



Pitfall! This is because each Set begins by zeroing the grid function - so any previous values are lost!

What happens if you use

   g.Set(cf, BND)

instead?


In [24]:
g = GridFunction(H1(mesh))
g.Set(cf, BND)
Draw(g, max=2, min=0)



Pitfall! We do not obtain the previous result because the keyword BND only sets the data in boundary regions marked as dirichlet.


In [26]:
g = GridFunction(H1(mesh, dirichlet='lin|lout'))
g.Set(cf, BND)
Draw(g, max=2, min=0)


### Unused dofs

Its important not to get confused with the ndof of spaces defined on subdomains. For example:


In [27]:
fes1 = H1(mesh, definedon="inner")
fes = H1(mesh)
fes1.ndof, fes.ndof

(11, 11)



Thus the ndofs are the same for spaces on one subdomain and the whole domain.

However, FreeDofs show that the real number of degrees of freedom for the subdomain space is much smaller:


In [28]:
print(fes1.FreeDofs())

0: 00011111000




One can also glean this information by examining CouplingType of each degrees of freedom of the space, which reveals that there are many unknowns of type COUPLING_TYPE.UNUSED_DOF.



In [29]:
for i in range(fes1.ndof):
    print(fes1.CouplingType(i))

COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
COUPLING_TYPE.UNUSED_DOF
